# Argo / GO-SHIP Comparison

This will serve as our working notebook, broken into our discussed working components below. 

## This cell for imports, constants

In [ ]:
# data processing
import numpy as np
import pandas as pd
from time import sleep

#data visualization
%matplotlib inline
import matplotlib.pyplot as plt
%matplotlib inline

# API convenience functions
from utilities_NSF_EC2022 import get_data_for_timeRange

import warnings
warnings.filterwarnings('ignore')

# set constants
URL_PREFIX = 'https://argovis-api.colorado.edu'
API_KEY = ''

## This cell for data loading/processing to xarray

## This cell for Data Visualization

In [ ]:
from argopy import DataFetcher as ArgoDataFetcher
ArgoSet = ArgoDataFetcher().float([6902746])

ds = ArgoSet.to_xarray()

## This cell for Data Interpolation/Comparison